In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df_adm = pd.read_csv('/Users/KexinHuang/Downloads/ADMISSIONS.csv')

In [4]:
len(df_adm.groupby(['SUBJECT_ID']))

46520

In [4]:
len(df_adm.groupby(['HADM_ID'])['HADM_ID'])

58976

In [5]:
df_adm.ADMITTIME = pd.to_datetime(df_adm.ADMITTIME, format = '%Y-%m-%d %H:%M:%S', errors = 'coerce')
df_adm.DISCHTIME = pd.to_datetime(df_adm.DISCHTIME, format = '%Y-%m-%d %H:%M:%S', errors = 'coerce')
df_adm.DEATHTIME = pd.to_datetime(df_adm.DEATHTIME, format = '%Y-%m-%d %H:%M:%S', errors = 'coerce')

In [6]:
df_adm = df_adm.sort_values(['SUBJECT_ID','ADMITTIME'])
df_adm = df_adm.reset_index(drop = True)

In [7]:
df_adm['NEXT_ADMITTIME'] = df_adm.groupby('SUBJECT_ID').ADMITTIME.shift(-1)

In [8]:
df_adm['NEXT_ADMISSION_TYPE'] = df_adm.groupby('SUBJECT_ID').ADMISSION_TYPE.shift(-1)

In [9]:
rows = df_adm.NEXT_ADMISSION_TYPE == 'ELECTIVE'
df_adm.loc[rows,'NEXT_ADMITTIME'] = pd.NaT
df_adm.loc[rows,'NEXT_ADMISSION_TYPE'] = np.NaN

In [10]:
df_adm = df_adm.sort_values(['SUBJECT_ID','ADMITTIME'])

df_adm[['NEXT_ADMITTIME','NEXT_ADMISSION_TYPE']] = df_adm.groupby(['SUBJECT_ID'])[['NEXT_ADMITTIME','NEXT_ADMISSION_TYPE']].fillna(method = 'bfill')

In [11]:
df_adm['DAYS_NEXT_ADMIT']=  (df_adm.NEXT_ADMITTIME - df_adm.DISCHTIME).dt.total_seconds()/(24*60*60)

In [13]:
df_adm1 = df_adm[['SUBJECT_ID','HADM_ID','ADMITTIME','DISCHTIME','DAYS_NEXT_ADMIT','NEXT_ADMITTIME','ADMISSION_TYPE','DEATHTIME']]

In [15]:
df_adm1['OUTPUT_LABEL'] = (df_adm1.DAYS_NEXT_ADMIT < 30).astype('int')

/anaconda/envs/pytorch/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [17]:
df_adm1['STAY_DAYS']=  (df_adm1.DISCHTIME - df_adm1.ADMITTIME).dt.total_seconds()/(24*60*60)

/anaconda/envs/pytorch/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [18]:
df_adm1.loc[df_adm1.DEATHTIME.notnull(), 'OUTPUT_LABEL'] = 0

/anaconda/envs/pytorch/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [19]:
len(df_adm1[df_adm1.DEATHTIME.notnull()])

5854

In [20]:
df_adm1.OUTPUT_LABEL.value_counts()

0    55790
1     3186
Name: OUTPUT_LABEL, dtype: int64

In [21]:
# drop newborn 
df_adm1 = df_adm1[df_adm1['ADMISSION_TYPE']!='NEWBORN']

In [22]:
df_adm1.OUTPUT_LABEL.value_counts()

0    48150
1     2963
Name: OUTPUT_LABEL, dtype: int64

In [23]:
#drop death
df_adm1 = df_adm1[df_adm1.DEATHTIME.isnull()]

In [24]:
df_adm1.OUTPUT_LABEL.value_counts()

0    42358
1     2963
Name: OUTPUT_LABEL, dtype: int64

In [28]:
len(df_adm1.groupby('SUBJECT_ID')['SUBJECT_ID'].unique())

34560

# Split

In [25]:
readmit_ID = df_adm1[df_adm1.OUTPUT_LABEL == 1].HADM_ID

In [26]:
not_readmit_ID = df_adm1[df_adm1.OUTPUT_LABEL == 0].HADM_ID

In [27]:
not_readmit_ID_use = not_readmit_ID.sample(n=len(readmit_ID), random_state=1)

In [28]:
id_val_test_t=readmit_ID.sample(frac=0.2,random_state=1)
id_val_test_f=not_readmit_ID_use.sample(frac=0.2,random_state=1)

In [29]:
id_train_t = readmit_ID.drop(id_val_test_t.index)

In [30]:
id_train_f = not_readmit_ID_use.drop(id_val_test_f.index)

In [31]:
id_val_t=id_val_test_t.sample(frac=0.5,random_state=1)
id_test_t=id_val_test_t.drop(id_val_t.index)

id_val_f=id_val_test_f.sample(frac=0.5,random_state=1)
id_test_f=id_val_test_f.drop(id_val_f.index)

In [33]:
(pd.Index(id_test_t).intersection(pd.Index(id_train_t))).values

array([], dtype=int64)

In [34]:
id_test = pd.concat([id_test_t, id_test_f])

In [35]:
len(id_test)

594

In [36]:
id_val = pd.concat([id_val_t, id_val_f])

In [37]:
len(id_val)

592

In [38]:
id_train = pd.concat([id_train_t, id_train_f])
train_id_label = pd.DataFrame(data = list(zip(id_train, [1]*len(id_train_t)+[0]*len(id_train_f))), columns = ['id','label'])

In [39]:
id_val = pd.concat([id_val_t, id_val_f])
val_id_label = pd.DataFrame(data = list(zip(id_val, [1]*len(id_val_t)+[0]*len(id_val_f))), columns = ['id','label'])

In [40]:
id_test = pd.concat([id_test_t, id_test_f])
test_id_label = pd.DataFrame(data = list(zip(id_test, [1]*len(id_test_t)+[0]*len(id_test_f))), columns = ['id','label'])

In [41]:
(pd.Index(train_id_label.id).intersection(pd.Index(test_id_label.id))).values

array([], dtype=int64)

In [42]:
len(train_id_label)

4740

In [120]:
train_id_label.to_csv('./good_datasets/good_train_id_label.csv')

In [121]:
test_id_label.to_csv('./good_datasets/good_test_id_label.csv')

In [122]:
val_id_label.to_csv('./good_datasets/good_val_id_label.csv')

## Discharge Processing

In [13]:
want=pd.read_csv('readmission_task.csv')
#discharge summary task dataset 

In [124]:
discharge_train = want[want.ID.isin(train_id_label.id)]

In [138]:
len(discharge_train)

24373

In [140]:
discharge_train.Label.value_counts()

1.0    13113
0.0    11260
Name: Label, dtype: int64

In [141]:
discharge_val = want[want.ID.isin(val_id_label.id)]

In [142]:
discharge_val.Label.value_counts()

1.0    1616
0.0    1424
Name: Label, dtype: int64

In [143]:
discharge_test = want[want.ID.isin(test_id_label.id)]

In [144]:
discharge_test.Label.value_counts()

1.0    1692
0.0    1366
Name: Label, dtype: int64

In [145]:
len(test_id_label)

594

In [133]:
df = pd.concat([not_readmit_ID_use, not_readmit_ID])
df = df.drop_duplicates(keep=False)

In [150]:
(pd.Index(df).intersection(pd.Index(not_readmit_ID_use))).values

array([], dtype=int64)

In [157]:
not_readmit_ID_more = df.sample(n=400, random_state=1)

In [158]:
dis_train = want[want.ID.isin(not_readmit_ID_more)]

In [159]:
(pd.Index(dis_train.ID).intersection(pd.Index(discharge_train.ID))).values

array([], dtype=float64)

In [160]:
len(dis_train)

1849

In [161]:
discharge_train_snippets = pd.concat([dis_train, discharge_train])

In [162]:
discharge_train_snippets = discharge_train_snippets.sample(frac=1, random_state=1).reset_index(drop=True)

In [165]:
discharge_train_snippets.Label.value_counts()

1.0    13113
0.0    13109
Name: Label, dtype: int64

In [166]:
discharge_train_snippets.to_csv('./good_datasets/discharge/train_snippets.csv')

In [167]:
discharge_val.to_csv('./good_datasets/discharge/val_snippets.csv')

In [168]:
discharge_test.to_csv('./good_datasets/discharge/test_snippets.csv')

## Now For Early Notes

In [ ]:
## get unactionable admits for Test
## Training 3 days

In [169]:
want_early = pd.read_csv('less_3_days_notes.csv')
#early notes fold 1

In [171]:
early_train = want_early[want_early.ID.isin(train_id_label.id)]

In [173]:
early_train.Label.value_counts()

1.0    23902
0.0    19809
Name: Label, dtype: int64

In [191]:
not_readmit_ID_more = df.sample(n=500, random_state=1)

In [192]:
ear_train = want_early[want_early.ID.isin(not_readmit_ID_more)]

In [193]:
ear_train.Label.value_counts()

0.0    4084
Name: Label, dtype: int64

In [194]:
early_train_snippets = pd.concat([ear_train, early_train])

In [199]:
(pd.Index(ear_train.ID).intersection(pd.Index(early_train.ID))).values

array([], dtype=float64)

In [202]:
early_train_snippets = early_train_snippets.sample(frac=1, random_state=1).reset_index(drop=True)

In [212]:
early_train_snippets.to_csv('./good_datasets/3days/train_snippets.csv')

In [205]:
early_val = want_early[want_early.ID.isin(val_id_label.id)]

In [206]:
early_val.Label.value_counts()

1.0    2986
0.0    2788
Name: Label, dtype: int64

In [214]:
early_val.to_csv('./good_datasets/3days/val_snippets.csv')

In [215]:
actionable_ID_3days = df_adm1[df_adm1['STAY_DAYS'] >= 3].HADM_ID

In [223]:
test_actionable_id_label = test_id_label[test_id_label.id.isin(actionable_ID_3days)]

In [224]:
early_test = want_early[want_early.ID.isin(test_actionable_id_label.id)]

In [225]:
early_test.Label.value_counts()

1.0    2904
0.0    2537
Name: Label, dtype: int64

In [226]:
early_test.to_csv('./good_datasets/3days/test_snippets.csv')

## Now for two days test set

In [236]:
want_2days = pd.read_csv('less_2_days_notes.csv')

In [237]:
actionable_ID_2days = df_adm1[df_adm1['STAY_DAYS'] >= 2].HADM_ID

In [238]:
test_actionable_id_label_2days = test_id_label[test_id_label.id.isin(actionable_ID_2days)]

In [245]:
early_test_2days = want_2days[want_2days.ID.isin(test_actionable_id_label_2days.id)]

In [246]:
early_test_2days.Label.value_counts()

1.0    2197
0.0    1857
Name: Label, dtype: int64

In [247]:
early_test_2days.to_csv('./good_datasets/2days/test_snippets.csv')

## To get other 4 fold Data

In [88]:
#fold 1: id_test, id_val, id_train
except_fold1 = pd.concat([not_readmit_ID_use, readmit_ID, id_test, id_val])

In [89]:
except_fold1 = except_fold1.drop_duplicates(keep=False)

In [90]:
except_fold1_f = (pd.Index(except_fold1).intersection(pd.Index(readmit_ID))).values
except_fold1_t = (pd.Index(except_fold1).intersection(pd.Index(not_readmit_ID_use))).values

In [91]:
len(except_fold1)

4740

In [92]:
len(except_fold1_f)

2370

In [93]:
folds = except_fold1.reset_index(drop=True)

In [94]:
from sklearn.model_selection import KFold
kf = KFold(n_splits = 4, shuffle = True, random_state = 1)
kf.get_n_splits(except_fold1_f)

4

In [95]:
re={}
not_re={}
for i, (train_index, test_index) in  enumerate(kf.split(except_fold1_f)):
    #print("TRAIN:", train_index) 
    #print("TEST:", test_index)
    not_re[i] = except_fold1_f[test_index]

In [108]:
len(not_re[2])

592

In [98]:
kf.get_n_splits(except_fold1_t)
for i, (train_index, test_index) in  enumerate(kf.split(except_fold1_t)):
    #print("TRAIN:", train_index) 
    #print("TEST:", test_index)
    re[i] = except_fold1_t[test_index]

In [107]:
sum(df_adm1[df_adm1.HADM_ID.isin(not_re[2])].OUTPUT_LABEL)

592

In [112]:
np.intersect1d(re[0],re[3])

array([], dtype=int64)

In [121]:
val_id_folds={}
test_id_folds={}

for i in range(4):
    ind_r = pd.Series(re[i]).sample(frac=0.5,random_state=1)
    d = pd.concat([ind_r, pd.Series(re[i])])
    ind_r1 = d.drop_duplicates(keep=False)
    ind_nr = pd.Series(not_re[i]).sample(frac=0.5,random_state=1)
    d = pd.concat([ind_nr, pd.Series(not_re[i])])
    ind_nr1 = d.drop_duplicates(keep=False)
    
    val_id_folds[i] = pd.concat([ind_r, ind_nr]).values
    test_id_folds[i] = pd.concat([ind_r1, ind_nr1]).values

In [124]:
np.intersect1d(test_id_folds[0],val_id_folds[0])

array([], dtype=int64)

In [125]:
sum(df_adm1[df_adm1.HADM_ID.isin(test_id_folds[0])].OUTPUT_LABEL)

297

In [127]:
train_id_folds={}
for i in range(4):
    all_index = pd.concat([not_readmit_ID_use, readmit_ID, pd.Series(test_id_folds[i]), pd.Series(val_id_folds[i])])
    train_id_folds[i] = all_index.drop_duplicates(keep=False).values                             

In [132]:
np.intersect1d(test_id_folds[3],train_id_folds[3])

array([], dtype=int64)

## folds discharge

# adjust the i in the following block to get different fold. Here, i = 0 means fold2, i=1 means fold 3, ... because we get fold 1 without using the Kfold library

In [176]:
want=pd.read_csv('readmission_task.csv')
df = pd.concat([not_readmit_ID_use, not_readmit_ID])
df = df.drop_duplicates(keep=False)

i=3
discharge_train = want[want.ID.isin(train_id_folds[i])]
discharge_val = want[want.ID.isin(val_id_folds[i])]
discharge_test = want[want.ID.isin(test_id_folds[i])]

discharge_train.Label.value_counts()

1.0    13182
0.0    11166
Name: Label, dtype: int64

In [177]:
discharge_test.Label.value_counts()

1.0    1547
0.0    1456
Name: Label, dtype: int64

In [180]:
dis_train = want[want.ID.isin(df.sample(n=437, random_state=1))]
len(dis_train)

2015

In [181]:
(pd.Index(dis_train.ID).intersection(pd.Index(discharge_train.ID))).values


array([], dtype=float64)

In [182]:
discharge_train_snippets = pd.concat([dis_train, discharge_train])
discharge_train_snippets = discharge_train_snippets.sample(frac=1, random_state=1).reset_index(drop=True)

In [183]:
file = './good_datasets/fold'+str(i+2)+'/discharge/train_snippets.csv'
discharge_train_snippets.to_csv(file)

In [184]:
file = './good_datasets/fold'+str(i+2)+'/discharge/val_snippets.csv'
discharge_val.to_csv(file)
file = './good_datasets/fold'+str(i+2)+'/discharge/test_snippets.csv'
discharge_test.to_csv(file)

## folds Early Notes

In [186]:
want_early = pd.read_csv('less_3_days_notes.csv')

In [200]:
want_2days = pd.read_csv('less_2_days_notes.csv')
actionable_ID_3days = df_adm1[df_adm1['STAY_DAYS'] >= 3].HADM_ID
actionable_ID_2days = df_adm1[df_adm1['STAY_DAYS'] >= 2].HADM_ID

In [259]:
i=3
early_train = want_early[want_early.ID.isin(train_id_folds[i])]
early_val = want_early[want_early.ID.isin(val_id_folds[i])]
test_actionable_id_3days = pd.Series(test_id_folds[i])[pd.Series(test_id_folds[i]).isin(actionable_ID_3days)]
early_test = want_early[want_early.ID.isin(test_actionable_id_3days)]

early_train.Label.value_counts()

1.0    23238
0.0    20071
Name: Label, dtype: int64

In [260]:
early_test.Label.value_counts()

1.0    2960
0.0    2486
Name: Label, dtype: int64

In [263]:
ear_train = want_early[want_early.ID.isin(df.sample(n=400, random_state=1))]
len(ear_train)

3161

In [264]:
early_train_snippets = pd.concat([ear_train, early_train])
early_train_snippets = early_train_snippets.sample(frac=1, random_state=1).reset_index(drop=True)

In [266]:
early_train_snippets.Label.value_counts()

1.0    23238
0.0    23232
Name: Label, dtype: int64

In [267]:
file = './good_datasets/fold'+str(i+2)+'/3days/train_snippets.csv'
early_train_snippets.to_csv(file)

In [268]:
file = './good_datasets/fold'+str(i+2)+'/3days/val_snippets.csv'
early_val.to_csv(file)
file = './good_datasets/fold'+str(i+2)+'/3days/test_snippets.csv'
early_test.to_csv(file)

In [269]:
(pd.Index(early_val.TEXT).intersection(pd.Index(early_train_snippets.TEXT))).values

array([], dtype=object)

In [270]:
test_actionable_id_2days = pd.Series(test_id_folds[i])[pd.Series(test_id_folds[i]).isin(actionable_ID_2days)]

In [271]:
len(test_actionable_id_2days)

555

In [272]:
early_test_2days = want_2days[want_2days.ID.isin(test_actionable_id_2days)]
early_test_2days.Label.value_counts()

1.0    2242
0.0    1848
Name: Label, dtype: int64

In [273]:
early_test_2days.to_csv('./good_datasets/fold'+str(i+2)+'/2days/test_snippets.csv')